In [618]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import numpy.random
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)

In [656]:
# 데이터 로드

train_data =pd.read_csv('train.csv', index_col=0)
test_data =pd.read_csv('test_x.csv', index_col=0)
submission=pd.read_csv('sample_submission.csv', index_col=0)  

In [657]:
# familysize 이상치 제거

train_data = train_data[train_data['familysize'] < 30]
print('이상치 제거 후:',train_data.shape)

이상치 제거 후: (45524, 77)


In [658]:
# 검사 소요시간은 로그 변환해서도 넣어봤으나 성능에 큰 영향이 없어 제거, hand도 제거

drop_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE'
             , 'hand']

In [659]:
# 명목형 변수 타입 변경


train_data['education']=train_data['education'].astype('str')
test_data['education']=test_data['education'].astype('str')

train_data['engnat']=train_data['engnat'].astype('str')
test_data['engnat']=test_data['engnat'].astype('str')

train_data['urban']=train_data['urban'].astype('str')
test_data['urban']=test_data['urban'].astype('str')

train_data['married']=train_data['married'].astype('str')
test_data['married']=test_data['married'].astype('str')

In [660]:
train_y=train_data['voted']
train_y = 2 - train_y.to_numpy()
train_data.drop('voted',axis=1,inplace=True)

In [661]:
train_data.drop(drop_list,axis=1,inplace=True)
test_data.drop(drop_list,axis=1,inplace=True)

### TIPI score 계산 
### 출처: https://www.psychologytoday.com/us/blog/darwins-subterranean-world/201810/take-quick-personality-test

In [663]:
tp_list=[i for i in train_data.columns if 'tp' in i]


In [664]:
#tp_score 계산 위해 무응답을 결측치로


train_data.loc[train_data['tp01']==7,'tp01']=np.nan
train_data.loc[train_data['tp02']==7,'tp02']=np.nan
train_data.loc[train_data['tp03']==7,'tp03']=np.nan
train_data.loc[train_data['tp04']==7,'tp04']=np.nan
train_data.loc[train_data['tp05']==7,'tp05']=np.nan
train_data.loc[train_data['tp06']==7,'tp06']=np.nan
train_data.loc[train_data['tp07']==7,'tp07']=np.nan
train_data.loc[train_data['tp08']==7,'tp08']=np.nan
train_data.loc[train_data['tp09']==7,'tp09']=np.nan
train_data.loc[train_data['tp10']==7,'tp10']=np.nan

In [665]:
test_data.loc[test_data['tp01']==7,'tp01']=np.nan
test_data.loc[test_data['tp02']==7,'tp02']=np.nan
test_data.loc[test_data['tp03']==7,'tp03']=np.nan
test_data.loc[test_data['tp04']==7,'tp04']=np.nan
test_data.loc[test_data['tp05']==7,'tp05']=np.nan
test_data.loc[test_data['tp06']==7,'tp06']=np.nan
test_data.loc[test_data['tp07']==7,'tp07']=np.nan
test_data.loc[test_data['tp08']==7,'tp08']=np.nan
test_data.loc[test_data['tp09']==7,'tp09']=np.nan
test_data.loc[test_data['tp10']==7,'tp10']=np.nan

In [666]:
#평균값으로 대체

train_data=train_data.fillna(train_data.mean())
test_data=test_data.fillna(test_data.mean())

In [667]:
#역변환

train_data[tp_list]=7-train_data[tp_list]
test_data[tp_list]=7-test_data[tp_list]

In [668]:
#tp score 공식대로 계산해 5가지 점수 생성

train_data['tp_ex']=(train_data['tp01']+7-(train_data['tp06']))/2
train_data['tp_ag']=(7-(train_data['tp02'])+train_data['tp07'])/2
train_data['tp_co']=(train_data['tp03']+7-(train_data['tp08']))/2
train_data['tp_em']=(7-(train_data['tp04'])+train_data['tp09'])/2
train_data['tp_op']=(train_data['tp05']+7-(train_data['tp10']))/2

In [669]:
test_data['tp_ex']=(test_data['tp01']+7-(test_data['tp06']))/2
test_data['tp_ag']=(7-(test_data['tp02']+test_data['tp07']))/2
test_data['tp_co']=(test_data['tp03']+7-(test_data['tp08']))/2
test_data['tp_em']=(7-(test_data['tp04']+test_data['tp09']))/2
test_data['tp_op']=(test_data['tp05']+7-(test_data['tp10']))/2

### 마키아벨리니즘 점수 계산, 코드 공유의 jhl님 글 활용하였습니다.
### 좋은 EDA 공유 감사드립니다!

In [670]:
#마키아벨리즘 점수계산을 위해 역변환 후 계산

Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    train_data[flip] = 6 - train_data[flip]
    
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    train_data[flip] = 6 - train_data[flip]
    
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    test_data[flip] = 6 - test_data[flip]
    
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    test_data[flip] = 6 - test_data[flip]
    
train_data['Mach_score'] = train_data[Answers].mean(axis = 1)
test_data['Mach_score'] = test_data[Answers].mean(axis = 1)

### 모델링

In [671]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

print(train_data.shape, test_data.shape)

(45524, 97) (11383, 97)


In [672]:
col=train_data.columns

train_x = train_data.to_numpy()
test_x = test_data.to_numpy()

train_x[:, :20] = (train_x[:, :20] - 3.) / 2.
test_x[:, :20] = (test_x[:, :20] - 3.) / 2
train_x[:, 20] = (train_x[:, 20] - 5.) / 5.
test_x[:, 20] = (test_x[:, 20] - 5.) / 5.
train_x[:, 21:31] = (train_x[:, 21:31] - 3.5) / 3.5
test_x[:, 21:31] = (test_x[:, 21:31] - 3.5) / 3.5

In [673]:
train_x=pd.DataFrame(train_x,columns=col)
test_x=pd.DataFrame(test_x,columns=col)

### TPOT Oprimizer

#### pycaret, h2o, 등 automl 방법을 찾아보다가  tpot을 사용해봤습니다.

In [674]:
from sklearn.model_selection import train_test_split

In [675]:
X_train,X_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.3,shuffle=True,stratify=train_y,random_state=123)

In [213]:
import tpot
pipeline_optimizer = tpot.TPOTClassifier(generations=20, 
                                         population_size=50, 
                                         cv=5,scoring='roc_auc',verbosity=2,early_stop=130,n_jobs=20) 
pipeline_optimizer.fit(X_train, y_train) 
print(pipeline_optimizer.score(X_test, y_test))


Generation 1 - Current best internal CV score: 0.7678834423260932

Generation 2 - Current best internal CV score: 0.7684363467228763

Generation 3 - Current best internal CV score: 0.7686594432249432

Generation 4 - Current best internal CV score: 0.7686594432249432

Generation 5 - Current best internal CV score: 0.7686981670872596

Generation 6 - Current best internal CV score: 0.7689514588842352

Generation 7 - Current best internal CV score: 0.769418843546107

Generation 8 - Current best internal CV score: 0.769418843546107

Generation 9 - Current best internal CV score: 0.7707555665105776

Generation 10 - Current best internal CV score: 0.7707555665105776

Generation 11 - Current best internal CV score: 0.7707555665105776

Generation 12 - Current best internal CV score: 0.7707555665105776

Generation 13 - Current best internal CV score: 0.7707744411455462

Generation 14 - Current best internal CV score: 0.7707744411455462

Generation 15 - Current best internal CV score: 0.77077444

### NN - Junho Sun님께서 공유해주신 모델 활용하였습니다. 덕분에 성능이 크게 향상됐습니다.
### 이번 대회 솔루션 모델이라고 할 수 있겠네요 감사합니다!

In [587]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_x = train_x.to_numpy()
test_x = test_x.to_numpy()

train_y = torch.tensor(train_y, dtype=torch.float32)
train_x = torch.tensor(train_x, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
train_len, test_len = len(train_x), len(test_x)

N_MODEL = 15
N_EPOCH = 30
BATCH_SIZE = 128
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((11383, 1), dtype=np.float32)

for no in range(N_MODEL):

    train_loader = DataLoader(TensorDataset(train_x, train_y),
                              shuffle=True, drop_last=True, **LOADER_PARAM)
    test_loader = DataLoader(TensorDataset(test_x, torch.zeros((test_len,), dtype=torch.float32)),
                             shuffle=False, drop_last=False, **LOADER_PARAM)
    model = nn.Sequential(
        nn.Dropout(0.05),
        nn.Linear(97, 96, bias=False),
        nn.LeakyReLU(0.05, inplace=True),
        nn.Dropout(0.5),
        nn.Linear(96, 36, bias=False),
        nn.ReLU(inplace=True),
        nn.Linear(36, 1)
    ).to(DEVICE)
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=4e-2)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=N_EPOCH // 4, eta_min=1.2e-5)

    model.train()
    for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(no + 1, N_MODEL)):
        for idx, (xx, yy) in enumerate(train_loader):
            optimizer.zero_grad()
            xx, yy = xx.to(DEVICE), yy.to(DEVICE)
            pred = model(xx).squeeze()
            loss = criterion(pred, yy)
            loss.backward()
            optimizer.step()
            scheduler.step(epoch + idx / len(train_loader))

    model.eval()
    with torch.no_grad():
        for idx, (xx, _) in enumerate(test_loader):
            xx = xx.to(DEVICE)
            pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
            prediction[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                += pred[:, :] / N_MODEL

15/15: 100%|██████████| 30/30 [01:10<00:00,  2.34s/it]


### LGBM, 특별한 튜닝은 못하고 다른 모델 결과와 앙상블했습니다.

In [579]:
import lightgbm as lgbm

params={
 'params': {'bagging_fraction':0.95,
  'colsample_bytree': 0.78,
  'feature_fraction': 0.95,
  'lambda_l1': 0.7,
  'lambda_l2': 0.6,
  'learning_rate': 0.01,
  'max_depth': 6.0,
  'min_child_weight': 33.0,
  'num_leaves': 380.0}}

#파라미터 적용
fit_lgbm = lgbm.LGBMClassifier(learning_rate=params['params']['learning_rate'],
                               num_leaves = int(round(params['params']['num_leaves'])),
                               max_depth = int(round(params['params']['max_depth'])),
                               min_child_weight = int(round(params['params']['min_child_weight'])),
                               colsample_bytree=params['params']['colsample_bytree'],
                               feature_fraction = max(min(params['params']['feature_fraction'], 1), 0),
                               bagging_fraction = max(min(params['params']['bagging_fraction'], 1), 0),
                               lambda_l1 = params['params']['lambda_l1'],
                               lambda_l2 = params['params']['lambda_l2'],
                               n_estimators=10000
                               )
evals = [(X_test, y_test)]

In [580]:
import warnings
warnings.filterwarnings('ignore')
model2 = fit_lgbm.fit(X_train,y_train.argmax(1),eval_set=evals,verbose=True,eval_metric='auc',early_stopping_rounds=500)

[1]	valid_0's auc: 0.755593	valid_0's binary_logloss: 0.686604
Training until validation scores don't improve for 500 rounds
[2]	valid_0's auc: 0.757182	valid_0's binary_logloss: 0.684528
[3]	valid_0's auc: 0.75551	valid_0's binary_logloss: 0.682495
[4]	valid_0's auc: 0.754541	valid_0's binary_logloss: 0.680494
[5]	valid_0's auc: 0.754444	valid_0's binary_logloss: 0.678528
[6]	valid_0's auc: 0.754073	valid_0's binary_logloss: 0.676602
[7]	valid_0's auc: 0.754143	valid_0's binary_logloss: 0.674726
[8]	valid_0's auc: 0.753943	valid_0's binary_logloss: 0.672887
[9]	valid_0's auc: 0.754172	valid_0's binary_logloss: 0.671074
[10]	valid_0's auc: 0.755062	valid_0's binary_logloss: 0.669277
[11]	valid_0's auc: 0.754846	valid_0's binary_logloss: 0.667522
[12]	valid_0's auc: 0.755095	valid_0's binary_logloss: 0.66581
[13]	valid_0's auc: 0.75487	valid_0's binary_logloss: 0.664119
[14]	valid_0's auc: 0.75512	valid_0's binary_logloss: 0.662452
[15]	valid_0's auc: 0.754741	valid_0's binary_logloss: 

### Submission

In [581]:
pred_lgb=model2.predict_proba(test_x)[:,0]

In [582]:
pred_tpot=pipeline_optimizer.predict_proba(test_x)[:,0]

In [615]:
submission=pd.read_csv('sample_submission.csv',index_col=0)
submission['voted'] = (prediction[:,0]-1)*0.8+(pred_tpot*0.1)+(pred_lgb*0.1)
submission.to_csv('submission.csv')